# MIT BIH heart beat classification model for heartbeats


The libary used to collect features from raw ECG data is the neurokit.p library which is used in this project to find the locations of important parts of each ecg wave including r-peak, p-peak,q-peak,r-onset,p-onset,p-offset,q-onset,q-offset. Being as this library does not come with python it must be installed to use this model.

In [7]:
pip install neurokit.py

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement neurokit.py
ERROR: No matching distribution found for neurokit.py


In [8]:
import numpy as np
import pandas as pd
import csv
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import tree
%run Functions.ipynb
%run FeaturesFunctions.ipynb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

loaded functions


First we need to create a training file with all of the annotated files in order. the files range from 100-235 not inclusive, the file 113 does not work with the Neurokit library so it was excluded from the model training.
File name: 1 csv for feature extraction

In [9]:

AnnotatedFileNames=[*range(100,125)]+[*range(200,235)]
remove=[110,113,120,204,206,211,216,218,224,225,226,227,229]
for i in remove:
    AnnotatedFileNames.remove(i)
Training = pd.DataFrame(columns = ['File','Result', 'Sample#'] )

for i in AnnotatedFileNames:
    temp= pd.DataFrame(columns = Training.columns)

    FileName ="mitbih_database/" + str(i)+'annotations.txt'
    annotations = mlbeats.df_from_txt(FileName)
    temp['Sample#']=annotations['Sample#']
    temp['Result']=annotations['Result']
    temp['File']= i
    
    Training = pd.concat([Training, temp], axis=0)
    

Adding neurokit data and aux data to training file.ipynbThen using the nerokit library we can add feature locations to set features in the data, Filename: Adding neurokit data and aux data to training file.ipynb

In [ ]:
testFeatures=pd.DataFrame()
for file in AnnotatedFileNames:
    DATA= pd.DataFrame(np.nan, index=range(1), columns=testFeatures.columns)
    testFeatures=pd.concat([testFeatures,DATA])
    testFeatures=pd.concat([testFeatures,Bryson.NerokitExtraction(file)])


extracting File 100


C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\epochs\eventrelated_utils.py:81: NeuroKitWarning: Input does not have an `ECG_Rate` column. Will skip all rate-related features.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:147: NeuroKitWarning: Input does not have an `ECG_Phase_Artrial` or `ECG_Phase_Ventricular` column. Will not indicate whether event onset concurs with cardiac phase.
  warn(
C:\Users\bryso\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:170: NeuroKitWarning: Input does not have an `ECG_Quality` column. Quality of the signal is not computed.
  warn(


We must then add the test features to the previously created training file.

In [ ]:
#NewTraining=pd.concat([NewTraining,NewFeatures],axis=1)
Training = Training.reset_index(drop=True)
testFeatures=testFeatures.reset_index(drop=True)
Training=pd.concat([Training,testFeatures],axis=1)
Training = Training.reset_index()
Training.head()

Due to the large amount of NaN values in the ECG_T_Peaks, ECG_T_Onsets ,ECG_T_Offsets ,and ECG_R_Offsets columns they are removed from the data.

In [ ]:

Training = Training.drop(["ECG_T_Peaks","ECG_T_Onsets","ECG_T_Offsets","ECG_R_Offsets"],axis='columns')
print(len(Training))
Training.dropna(inplace=True)
print(len(Training))

Now, Interval data is collected from the time data to get useful information to build the model on.

In [ ]:
Interval = pd.DataFrame(columns=["File","Abnormal","Result","R-R Interval","R Height","R_Onset-Rpeak","Q-Q Interval","Q Height","P_Onset-P_Offset","P Height","P-P Interval","P_Onset-Ppeak"])
RRLIST=np.array([])
QQLIST=np.array([])
R_Height=np.array([])
Q_Height=np.array([]) 
P_Height=np.array([])
Ron_RP=np.array([])
Pon_PP=np.array([])
Pon_Poff=np.array([])
PPLIST=np.array([])
Abnormal=np.array([])
Result=np.array([])
FILES=np.array([])
y=2
FileNumber=Training["File"][y]
RAWFile= pd.read_csv("mitbih_database/"+str(FileNumber)+".csv")


for i in Training["index"][:-1]:
    if (i-y>1):
        y=i
        continue
    if Training["Result"][i]=='N':
        Abnormal=np.append(Abnormal,0)
    else:
        Abnormal=np.append(Abnormal,1)
    if (Training["File"][i] != FileNumber):
        print("Reading File",FileNumber)
        FileNumber=Training["File"][i]
        RAWFile= pd.read_csv("mitbih_database/"+str(FileNumber)+".csv")
        
    RRLIST=np.append(RRLIST,Training["ECG_R_Peaks"][i]-Training["ECG_R_Peaks"][y])
    QQLIST=np.append(QQLIST,Training["ECG_Q_Peaks"][i]-Training["ECG_Q_Peaks"][y])
    PPLIST=np.append(PPLIST,Training["ECG_P_Peaks"][i]-Training["ECG_P_Peaks"][y])
    R_Height=np.append(R_Height,RAWFile[RAWFile.columns[1]][Training["ECG_R_Peaks"][i]])#
    Q_Height=np.append(Q_Height,RAWFile[RAWFile.columns[1]][Training["ECG_Q_Peaks"][i]])#
    P_Height=np.append(P_Height,RAWFile[RAWFile.columns[1]][Training["ECG_P_Peaks"][i]])#
    Ron_RP=np.append(Ron_RP,Training["ECG_R_Peaks"][i]-Training["ECG_R_Onsets"][i])
    Pon_PP=np.append(Pon_PP,Training["ECG_P_Peaks"][i]-Training["ECG_P_Onsets"][i])
    Pon_Poff=np.append(Pon_Poff,Training["ECG_P_Offsets"][i]-Training["ECG_P_Onsets"][i])
    Result=np.append(Result,Training["Result"][i])
    FILES=np.append(FILES,FileNumber)
    
    y=i

Interval["R-R Interval"]=RRLIST
Interval["Q-Q Interval"]=QQLIST
Interval["P-P Interval"]=PPLIST
Interval["R_Onset-Rpeak"]=Ron_RP
Interval["P_Onset-Ppeak"]=Pon_PP
Interval["P_Onset-P_Offset"]=Pon_Poff
Interval["R Height"]=R_Height#
Interval["Q Height"]=Q_Height#
Interval["P Height"]=P_Height#
Interval["File"]=FILES
Interval["Abnormal"]=Abnormal
Interval["Result"]=Result

print("Length of DataFrame",len(Interval))
Interval.head(10)



Now a decision Tree classification model is created to descern between normal and abrnormal heartbeats, the model is then scored using regular scoring techniques along with the AUC score 

In [ ]:
model1 = tree.DecisionTreeClassifier()

inputs = Interval.drop(["Result","Abnormal"],axis='columns')
Target = Interval[["Abnormal"]]

x_train1, x_test1, y_train1, y_test1 = train_test_split(inputs, Target,test_size=0.3)

model1.fit(X=x_train1,y=y_train1)
print(model1.score(x_test1,y_test1),"Regular")
print(roc_auc_score(y_test1, model1.predict(x_test1)),"AUC1\n")

In [ ]:
_ = tree.plot_tree(model1)